<a href="https://colab.research.google.com/github/flee2-trinity/Deep-Learning-in-Sports-Result-Prediction-/blob/main/PlayByPlay1819.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/2021 Fall/Thesis I'

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import datetime

def findFiles(path): 
  return glob.glob(path)

In [ ]:
import csv

filename = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba 18-19/NatStat-NBA2019-Play-by-Play.csv'
with open(filename, newline='') as f:
      reader = csv.reader(f)
      #print(list(reader)[0])
      """
      ['GameDay', 'GameID', 'Sort', 'Period', 'Time', 'Visitor', 
      'VisitorID', 'ScoreVis', 'Home', 'HomeID', 'ScoreHome', 
      'Team', 'TeamID', 'TeamAbbrev', 'OppID', 'Opp', 'Diff', 
      'PlayerID', 'Description', 'ScoringPlay', 'Points', 
      'Distance', 'Tags', 'FieldHome', 'FieldVis']
      """
      next(reader)
      rawDataL = list(reader)
      """
      ['2018-10-16', '985248', '1-1760', '1', '12:00', 'Philadelphia 76ers', 
      '28', '0', 'Boston Celtics', '5', '0', 'Philadelphia 76ers', '28', 'PHI', 
      '5', 'Boston Celtics', '0', '0', 
      'Jump Ball: Joel Embiid vs. Al Horford -- Ben Simmons gains possession', 
      '', '0', '0', 'JUMP', '', '']
      """
print(rawDataL[0])

['2018-10-16', '985248', '1-1760', '1', '12:00', 'Philadelphia 76ers', '28', '0', 'Boston Celtics', '5', '0', 'Philadelphia 76ers', '28', 'PHI', '5', 'Boston Celtics', '0', '0', 'Jump Ball: Joel Embiid vs. Al Horford -- Ben Simmons gains possession', '', '0', '0', 'JUMP', '', '']


In [ ]:
for i in range(5):
  print(rawDataL[i])
"""
      ['GameDay', 'GameID', 'Sort', 'Period', 'Time', 'Visitor', 
      'VisitorID', 'ScoreVis', 'Home', 'HomeID', 'ScoreHome', 
      'Team', 'TeamID', 'TeamAbbrev', 'OppID', 'Opp', 'Diff', 
      'PlayerID', 'Description', 'ScoringPlay', 'Points', 
      'Distance', 'Tags', 'FieldHome', 'FieldVis']
      """

['2018-10-16', '985248', '1-1760', '1', '12:00', 'Philadelphia 76ers', '28', '0', 'Boston Celtics', '5', '0', 'Philadelphia 76ers', '28', 'PHI', '5', 'Boston Celtics', '0', '0', 'Jump Ball: Joel Embiid vs. Al Horford -- Ben Simmons gains possession', '', '0', '0', 'JUMP', '', '']
['2018-10-16', '985248', '1-1820', '1', '11:40', 'Philadelphia 76ers', '28', '0', 'Boston Celtics', '5', '0', 'Philadelphia 76ers', '28', 'PHI', '5', 'Boston Celtics', '0', '0', 'Robert Covington misses a 3-point jump shot from 28 feet out', '', '0', '28', '3FA', '', '']
['2018-10-16', '985248', '1-1820', '1', '11:40', 'Philadelphia 76ers', '28', '0', 'Boston Celtics', '5', '0', 'Boston Celtics', '5', 'BOS', '28', 'Philadelphia 76ers', '0', '0', 'Celtics defensive rebound', '', '0', '0', 'REB|TRB', '', '']
['2018-10-16', '985248', '1-1845', '1', '11:15', 'Philadelphia 76ers', '28', '0', 'Boston Celtics', '5', '0', 'Boston Celtics', '5', 'BOS', '28', 'Philadelphia 76ers', '0', '329820', 'Jayson Tatum misses a 3

"\n      ['GameDay', 'GameID', 'Sort', 'Period', 'Time', 'Visitor', \n      'VisitorID', 'ScoreVis', 'Home', 'HomeID', 'ScoreHome', \n      'Team', 'TeamID', 'TeamAbbrev', 'OppID', 'Opp', 'Diff', \n      'PlayerID', 'Description', 'ScoringPlay', 'Points', \n      'Distance', 'Tags', 'FieldHome', 'FieldVis']\n      "

In [ ]:
import datetime

def readTime(ampm):
  t = 0
  tL = ampm.replace(' ',':').split(':')
  t += int(tL[0])*60
  t += int(tL[1])
  return t

seasonBegin = datetime.datetime(2018, 10, 16)
playoffsBegin = datetime.datetime(2019, 4, 13)
class Tags:
  def __init__(self):
    self.FA3 = False
    self.FM3 = False
    self.FGA = False
    self.FGM = False
    self.FTA = False
    self.FTM = False
    self.PF = False
    self.TRB = False
    self.ORB = False
    self.REB = False
    self.STL = False  
    self.TO = False
    self.JUMP = False
    self.BLK = False
    self.MISC = False
    self.AST = False
    self.SUB = False
    self.TIMEOUT = False
    self.Nothing = True
def readTags(TagsLine):
  tagC = Tags()
  tSplitted = TagsLine.split("|")
  tagC.Nothing = False
  for t in tSplitted:
    if t == "3FA":
      tagC.FA3 = True
    elif t == "3FM":
      tagC.FM3 = True
    else:
      setattr(tagC, t, True)
  return tagC
  
class aPlay:
  def __init__(self, GameDay, GameID, Sort, Period, Time, Visitor, 
      VisitorID, ScoreVis, Home, HomeID, ScoreHome, 
      Team, TeamID, TeamAbbrev, OppID, Opp, Diff, 
      PlayerID, Description, ScoringPlay, Points, 
      Distance, Tags, FieldHome, FieldVis):
    
    gdL = GameDay.split("-")
    self.GameDay = datetime.datetime(int(gdL[0]), int(gdL[1]), int(gdL[2]))
    self.GameID = int(GameID)
    self.Period = int(Period)
    self.Time = readTime(Time)
    self.Visitor = Visitor
    self.VisitorID = int(VisitorID)
    self.ScoreVis = int(ScoreVis)
    self.Home = Home
    self.HomeID = int(HomeID)
    self.ScoreHome = int(ScoreHome)
    self.Team = Team
    self.TeamID = int(TeamID)
    self.TeamAbbrev = TeamAbbrev
    self.OppID = int(OppID)
    self.Opp = Opp
    self.Diff = int(Diff)
    self.SDiff = self.ScoreHome - self.ScoreVis
    self.PlayerID = int(PlayerID)
    self.Description = Description
    self.ScoringPlay = (ScoringPlay == 'Y')
    self.Points = int(Points)
    self.Distance = int(Distance)
    self.TagsString = Tags
    self.FieldHome = FieldHome #No Value
    self.FieldVis = FieldVis #No Value
    self.Tags = readTags(Tags)
    if TeamID == HomeID:
      self.SameTeamI = 1
    else:
      self.SameTeamI = -1
    self.DayCount = (self.GameDay - seasonBegin).days
    self.Playoffs = (self.GameDay >= playoffsBegin)
    self.OT = int(self.Period == 0)
  def addGResult(self, aG):
    self.aGame = aG
  def printDay(self):
    print('Day', self.DayCount)


class Stats:
  def __init__(self):
    """
    MIN, PTS,FGM, FGA, 3FM, 3FA, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF
    """
    self.MIN = 0
    self.PTS = 0
    self.FGM = 0
    self.FGA = 0
    self.FM3 = 0
    self.FA3 = 0
    self.FTM = 0
    self.FTA = 0
    self.REB = 0
    self.AST = 0
    self.STL = 0
    self.BLK = 0
    self.OREB = 0
    self.TO = 0
    self.PF = 0
    self.Win = 0
    self.Loss = 0
    self.WinRate = 0
    self.WinBy = 0
  def update(self, MIN, PTS,FGM, FGA, FM3, FA3, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF):
    self.MIN = MIN
    self.PTS = PTS
    self.FGM = FGM
    self.FGA = FGA
    self.FM3 = FM3
    self.FA3 = FA3
    self.FTM = FTM
    self.FTA = FTA
    self.REB = REB
    self.AST = AST
    self.STL = STL
    self.BLK = BLK
    self.OREB = OREB
    self.TO = TO
    self.PF = PF
  def addWinRate(self, Win, Loss):
    self.Win = Win
    self.Loss = Loss
    self.WinRate = Win/(Win+Loss)
  
class aGame:
  def __init__(self, GameDay, GameID, TeamID, Team, OpponentID, Opponent, Location,
       Division, Conference, Playoffs, WinOrLoss, PlayerType):
    self.GameID = int(GameID)
    self.TeamID = int(TeamID)
    gdL = GameDay.split("-")
    self.GameDay = datetime.datetime(int(gdL[0]), int(gdL[1]), int(gdL[2]))
    #self.GameTime = GameTime
    self.DayCount = (self.GameDay - seasonBegin).days
    self.Team = Team
    self.OpponentID = int(OpponentID)
    self.Opponent = Opponent
    self.Home = Location == 'H'
    self.Division = Division
    self.Conference = Conference
    self.Playoffs = Playoffs == 'Y'
    self.WinOrLoss = WinOrLoss == 'W'
    self.WinID = int(self.WinOrLoss)
    self.LossID = int(not self.WinOrLoss)
    self.PlayerType = PlayerType
    self.TStats = Stats()
    self.OppStats = Stats()
    self.TStatsP = Stats()
    self.OppStatsP = Stats()


In [ ]:
def aLinetoAPlay(aline):
  aP = aPlay(aline[0], aline[1], aline[2], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11], aline[12], aline[13],
               aline[14], aline[15], aline[16], aline[17], aline[18], aline[19], aline[20],
               aline[21], aline[22], aline[23], aline[24])

  return aP
def aLinetoGame(aline):
  newG = aGame(aline[0], aline[1], aline[2], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11])
  for n in range(12, 27):
    aline[n] = int(aline[n])

  newG.TStats.update(aline[12], aline[13], aline[14], aline[15], aline[16], aline[17], 
                     aline[18], aline[19], aline[20], aline[21], aline[22], aline[23], 
                     aline[24], aline[25], aline[26])
  return newG

In [ ]:
def readLines(filename, forPlay = True):
  with open(filename, newline='') as f:
      reader = csv.reader(f)
      next(reader)
      rawDataL = list(reader)
  #print(rawDataL)
  #'ID', 'GameDay', 'GameTime', 'Home', 'HomeID', 'Visitor', 'VisitorID', 'ScoreVis', 
  #'ScoreHome', 'GameStatus', 'Venue', 'CityState', 'Neutral', 'Division', 'Conference', 'Playoffs'
  #print("RawDataLength ",len(rawDataL))
  data = []
  if forPlay:
    for oneLine in rawDataL:
      g = aLinetoAPlay(oneLine)
      data.append(g)
  else:
    for oneLine in rawDataL:
      g = aLinetoGame(oneLine)
      data.append(g)
  return data

In [ ]:
filenameTeam = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba 18-19/NatStat-NBA2019-Team_Statlines.csv'
allDataTeam = readLines(filenameTeam, forPlay = False)

In [ ]:
from copy import deepcopy
allData = readLines(filename)

In [ ]:
print(rawDataL[0])
print(allData[0].Tags.REB)
print(allData[0].Tags.JUMP)
print(len(allData))
print(len(allDataTeam))

['2018-10-16', '985248', '1-1760', '1', '12:00', 'Philadelphia 76ers', '28', '0', 'Boston Celtics', '5', '0', 'Philadelphia 76ers', '28', 'PHI', '5', 'Boston Celtics', '0', '0', 'Jump Ball: Joel Embiid vs. Al Horford -- Ben Simmons gains possession', '', '0', '0', 'JUMP', '', '']
False
True
543835
2580


In [ ]:
from collections import defaultdict
groups = defaultdict(list)

for o in allData:
    groups[o.GameID].append(o)

gList = list(groups.values())

In [ ]:
print(type(gList))
print(gList[0][0].GameID)
dataF = []
for gl in gList:
  xL = list(filter(lambda x: x.GameID == gl[0].GameID and x.Home, allDataTeam))
  x = xL[0]
  if(len(xL) != 1):
    print("gl[0].GameID")
  for g in gl:
    g.addGResult(x)
    #print(x.WinID)
    dataF.append(g)
print(len(dataF))
#for g in gList:


<class 'list'>
985248
543835


In [ ]:
def pick2GroupsG(allData, afterN = 120, range = 5):
  allData = list(filter(lambda g: not g.Playoffs, allData))
  nData = len(allData)
  test_daysAfterN = list(filter(lambda g: g.DayCount >= afterN, allData))
  test_daysAfterN = list(map(lambda g: g.DayCount, test_daysAfterN))
  test_days = list(set(test_daysAfterN))
  test_days.sort()
  test_listGroups = []
  train_listGroups = []
  for daysC in test_days[:range]:
    test_listGroups.append(list(filter(lambda g: g.DayCount == daysC, allData)))
    train_listGroups.append(list(filter(lambda g: g.DayCount < daysC, allData)))
  return (train_listGroups, test_listGroups)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def playtoTensor(p : aPlay):
  tensor = torch.zeros(1, 1, 110)
  """
  ['GameDay', 'GameID', 'Sort', 'Period', 'Time', 'Visitor', 
      'VisitorID', 'ScoreVis', 'Home', 'HomeID', 'ScoreHome', 
      'Team', 'TeamID', 'TeamAbbrev', 'OppID', 'Opp', 'Diff', 
      'PlayerID', 'Description', 'ScoringPlay', 'Points', 
      'Distance', 'Tags', 'FieldHome', 'FieldVis']

      self.FA3 = False
    self.FM3 = False
    self.FGA = False
    self.FGM = False
    self.FTA = False
    self.FTM = False
    self.PF = False

    self.TRB = False
    self.ORB = False
    self.REB = False
    self.STL = False  
    self.TO = False
    self.JUMP = False
    self.BLK = False
    self.MISC = False
    self.AST = False
    self.SUB = False
    self.TIMEOUT = False
      """
  tensor[0][0][p.VisitorID-1] = 1
  tensor[0][0][p.HomeID+37] = 1
  tensor[0][0][76] = int(p.aGame.TeamID == p.TeamID)
  tensor[0][0][77] = float(p.DayCount)/250
  tensor[0][0][78] = float(p.Period)/4
  tensor[0][0][79] = float(p.Time)/720
  tensor[0][0][80] = float(p.SDiff)/80
  tensor[0][0][81] = int(p.ScoringPlay)
  tensor[0][0][82] = float(p.Points)/3
  tensor[0][0][83] = int(p.Tags.FA3)
  tensor[0][0][84] = int(p.Tags.FM3)
  tensor[0][0][85] = int(p.Tags.FGA)
  tensor[0][0][86] = int(p.Tags.FGM)
  tensor[0][0][87] = int(p.Tags.FTA)
  tensor[0][0][88] = int(p.Tags.FTM)
  tensor[0][0][89] = int(p.Tags.PF)
  tensor[0][0][90] = int(p.Tags.TRB)
  tensor[0][0][91] = int(p.Tags.ORB)
  tensor[0][0][92] = int(p.Tags.REB)
  tensor[0][0][93] = int(p.Tags.STL)
  tensor[0][0][94] = int(p.Tags.TO)
  tensor[0][0][95] = int(p.Tags.JUMP)
  tensor[0][0][96] = int(p.Tags.BLK)
  tensor[0][0][97] = int(p.Tags.MISC)
  tensor[0][0][98] = int(p.Tags.AST)
  tensor[0][0][99] = int(p.Tags.SUB)
  tensor[0][0][100] = int(p.Tags.TIMEOUT)
  tensor[0][0][101] = int(p.Tags.Nothing)
  tensor[0][0][102] = int(p.TeamID == p.HomeID)
  tensor[0][0][103] = float(p.ScoreVis)/170
  tensor[0][0][104] = float(p.ScoreHome)/170
  tensor[0][0][105] = ((p.Period-1)*720+float(p.Time))/3000
  tensor[0][0][106] = float(p.Points)/4
  tensor[0][0][107] = p.OT
  tensor[0][0][108] = p.SameTeamI
  """
  tensor[0][0][78] = g.TStatsP.MIN/260
  tensor[0][0][79] = g.TStatsP.PTS/160
  tensor[0][0][80] = g.TStatsP.FGM/60
  tensor[0][0][81] = g.TStatsP.FGA/120
  tensor[0][0][82] = g.TStatsP.FM3/30
  tensor[0][0][83] = g.TStatsP.FTM/50
  tensor[0][0][84] = g.TStatsP.FTA/60
  tensor[0][0][85] = g.TStatsP.REB/80
  tensor[0][0][86] = g.TStatsP.AST/50
  tensor[0][0][87] = g.TStatsP.STL/20
  tensor[0][0][88] = g.TStatsP.BLK/20
  tensor[0][0][89] = g.TStatsP.OREB/50
  tensor[0][0][90] = g.TStatsP.TO/50
  tensor[0][0][91] = g.TStatsP.PF/50
  
  tensor[0][0][92] = g.TStatsP.Win/72
  tensor[0][0][93] = g.TStatsP.Loss/72
  tensor[0][0][94] = g.TStatsP.WinRate
  
  tensor[0][0][95] = g.OppStatsP.MIN/260
  tensor[0][0][96] = g.OppStatsP.PTS/160
  tensor[0][0][97] = g.OppStatsP.FGM/60
  tensor[0][0][98] = g.OppStatsP.FGA/120
  tensor[0][0][99] = g.OppStatsP.FM3/30
  tensor[0][0][100] = g.OppStatsP.FTM/50
  tensor[0][0][101] = g.OppStatsP.FTA/60
  tensor[0][0][102] = g.OppStatsP.REB/80
  tensor[0][0][103] = g.OppStatsP.AST/50
  tensor[0][0][104] = g.OppStatsP.STL/20
  tensor[0][0][105] = g.OppStatsP.BLK/20
  tensor[0][0][106] = g.OppStatsP.OREB/50
  tensor[0][0][107] = g.OppStatsP.TO/50
  tensor[0][0][108] = g.OppStatsP.PF/50

  tensor[0][0][109] = g.OppStatsP.Win/72
  tensor[0][0][110] = g.OppStatsP.Loss/72
  tensor[0][0][111] = g.OppStatsP.WinRate
  
  tensor[0][0][112] = g.TStatsP.WinBy/60
  tensor[0][0][113] = g.OppStatsP.WinBy/60
  """
  return tensor

def playstoTensorTrain(plays):
  p_tensor_list = []
  for p in plays:
    p_tensor_list.append((playtoTensor(p), p.aGame.WinID))
    #print(p.aGame.WinID)
    #print(g_tensor_list[0][1][0][0].item())
  return p_tensor_list
def playstoTensorTest(plays):
  p_tensor_list = []
  for p in plays:
    p_tensor_list.append((playtoTensor(p), p.aGame.WinID, p.Period, p.Time, p.SDiff, p.GameID, p.DayCount))
  return p_tensor_list

In [ ]:
import time
import math
import statistics
from statistics import mean
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def runModel(train_tlistGroups, test_tlistGroups):
  start = time.time()
  
  time_list = []
  print(len(train_tlistGroups), len(test_tlistGroups))
  for t in range(len(train_tlistGroups)):
    print('Model: ', t)
    model = nn.Sequential(nn.Linear(110, 64),
                          nn.Tanh(),
                          nn.Linear(64, 16),
                          nn.Tanh(),
                          nn.Linear(16, 2),
                          nn.LogSoftmax(dim=1)
                          #nn.Tanh(),
                          )
    #model = model.to(device=device)
    learning_rate = 0.005
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_fn = nn.NLLLoss()#meansquarerror
    n_epochs = 30
    for epoch in range(n_epochs):
      
      train_loader = torch.utils.data.DataLoader(train_tlistGroups[t], batch_size = 32, shuffle = True)
      for gs, labels in train_loader:
      #for g, label in train_tlistGroups[t]:
        #gs = gs.to(device=device)
        #labels = labels.to(device = device)

        #batch_size = g.shape[0]
        batch_size = gs.shape[0]
        outputs = model(gs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
      if (epoch+1)%10 == 0:
        print(timeSince(start))
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
        
    for gt, label, period, tm, diff, gID, dayC in test_tlistGroups[t]:
      outputs = model(gt.view(1, -1))
      #print('-')
      #print(outputs)
      _, predicted = torch.max(outputs, dim=1)
      #print(predicted)
      #print(label)
      time_list.append((int(predicted == label), period, tm, diff, gID, dayC))
  print(timeSince(start))
  return time_list

In [ ]:
def writeInPBP(newRecord):
  p1Name='/content/gdrive/My Drive/2021 Fall/Thesis I/Records/PBP1819GID.txt'
  p1file=open(p1Name)
  p1file_content = p1file.read()
  p1file=open(p1Name,"w")
  p1file.write(p1file_content)
  for label, period, tm, diff, gID, dayC in newRecord:
    p1file.write(str(label)+" "+ str(period) + " " + str(tm)+ " " + str(abs(diff))+  " " + str(gID)+ " " + str(dayC) + "\n")
  p1file.write("-\n")
  p1file=open(p1Name)
  p1file_content = p1file.read()
  #print(p1file_content)
  p1file.close()

In [ ]:
trainGs, testGs = pick2GroupsG(allData, afterN = 140, range = 10)

In [ ]:
train_tlistGroups = list(map(lambda group: playstoTensorTrain(group), trainGs))
test_tlistGroups = list(map(lambda group: playstoTensorTest(group), testGs))


In [ ]:
timeList = []
for i in range(1):
  print('Run %d' %i)
  writeInPBP(runModel( train_tlistGroups, test_tlistGroups))
  #writeIn(runModel(trainGs, testGs))

Run 0
10 10
Model:  0
1m 35s
Epoch: 9, Loss: 0.331425
3m 8s
Epoch: 19, Loss: 0.722927
4m 41s
Epoch: 29, Loss: 0.011758
Model:  1
6m 17s
Epoch: 9, Loss: 0.209137
7m 52s
Epoch: 19, Loss: 0.021905
9m 26s
Epoch: 29, Loss: 0.008264
Model:  2
11m 1s
Epoch: 9, Loss: 0.336417
12m 35s
Epoch: 19, Loss: 0.150905
14m 10s
Epoch: 29, Loss: 0.025294
Model:  3
15m 45s
Epoch: 9, Loss: 0.370902
17m 20s
Epoch: 19, Loss: 0.039016
18m 54s
Epoch: 29, Loss: 0.024289
Model:  4
20m 30s
Epoch: 9, Loss: 0.352816
22m 7s
Epoch: 19, Loss: 0.086230
23m 42s
Epoch: 29, Loss: 0.014520
Model:  5
25m 19s
Epoch: 9, Loss: 0.317948
26m 55s
Epoch: 19, Loss: 0.133803
28m 31s
Epoch: 29, Loss: 0.005287
Model:  6
30m 8s
Epoch: 9, Loss: 0.135404
31m 44s
Epoch: 19, Loss: 0.327760
33m 20s
Epoch: 29, Loss: 0.129369
Model:  7
34m 58s
Epoch: 9, Loss: 0.463993
36m 35s
Epoch: 19, Loss: 0.162860
38m 12s
Epoch: 29, Loss: 0.019565
Model:  8
39m 49s
Epoch: 9, Loss: 0.360541
41m 26s
Epoch: 19, Loss: 0.124194
43m 3s
Epoch: 29, Loss: 0.035620
